In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #CUAD를 사용하기 위한 device 설정
print(device)

In [ ]:
#데이터 변환 입력 이미지를 torch.FloatTensor로 변환
#N변환된 torch.FloatTensor를 Normalize한다 transforms.Normalize((mean),(std))
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4 #배치사이즈 hyperparameter 설정

#training data set 설정 
#root = str : data set의 루트 디렉토리 지정 인자
#train = true : tarining set에서 data set 구성 인자
#transform : 이미지 data를 변환하는 함수 설정 인자
#download : data set 다운로드 설정 인자
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
#training data set loading 설정
#trainset : load할 data set
#batch_size : 로드할 배치당 셈플 수
#shuffle : 각각의 epoch에서 data set를 다시 섞도록 설정
#nun_workers : data load에 사용할 하위 프로세스 수
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
#test data set 설정
#train = false : test set에서 data set 구성 인자
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
#test data set loading 설정
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
#클래스 10 설정
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__() #부모클래스(nn.Moudle)의 내용을 사용하기 위해서 super()사용
        self.conv1 = nn.Conv2d(3, 6, 5) #2D convolution (input channel size, out channel size, kernel_size)
        self.pool = nn.MaxPool2d(2, 2) #2D maxpooling (kernel_size, padding)
        self.conv2 = nn.Conv2d(6, 16, 5)  #2D convolution
        self.fc1 = nn.Linear(16 * 5 * 5, 120) #Linear transformation (input_features, output_features)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #conv1 relu pooling 계산 후 x 초기화
        x = self.pool(F.relu(self.conv2(x))) #conv2 relu pooling 계산 후 x 초기화
        x = torch.flatten(x, 1) #1차원 텐서로 변형
        x = F.relu(self.fc1(x)) #fully connected 1 relu 후 x 초기화
        x = F.relu(self.fc2(x)) #fully connected 2 relu 후 x 초기화
        x = self.fc3(x) ##fully connected 3 계산 후 x 초기화
        return x


net = Net().to(device) #완성된 모델 초기화 + GPU로 올리기위한 인자

In [8]:
criterion = nn.CrossEntropyLoss() #crossentropyloss 사용
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) #optimizer로 SGD사용 (parameter 설정, learning rate, momentum 설정)

In [ ]:
for epoch in range(2): #트레이닝 셋이 신경망을 통과한 횟수 지정 

    running_loss = 0.0 #loss 초기화
    for i, data in enumerate(trainloader, 0): #반복문, training data 에서 traninloader를 통해 순서대로 data 불러오기
        inputs, labels = data[0].to(device), data[1].to(device) #input data , label data 불러오기 + 설정된 device로 이동
        optimizer.zero_grad() #torch.Tensor의 기울기를 0으로 초기화
        outputs = net(inputs) #신경망 모델에 data 넣어 outpur 값 저장
        loss = criterion(outputs, labels) #output값과 labels값을 통한 loss 값 crossentropyloss로 계산
        loss.backward() #loss packprop
        optimizer.step() #optimizer 매개변수 업데이트 
        running_loss += loss.item() #loss값 저장
        if i % 2000 == 1999:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000)) #학습현황 표시
            running_loss = 0.0 #loss 초기화

print('Finished Training')

In [10]:
PATH = './cifar_net.pth' #학습한 parameter 값 저장 디렉토리 설정
torch.save(net.state_dict(), PATH) # parameter 값 저장

In [ ]:
net = Net().to(device) #모델 불러오기
net.load_state_dict(torch.load(PATH)) #학습한 parameter 불러오기

In [ ]:
correct = 0
total = 0

with torch.no_grad(): # 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요없음
    for data in testloader: #test data set에서 testloader롤 통한 data 불러오기
        images, labels = data[0].to(device), data[1].to(device) #불러온 data inages, labels 에 지정 및 설정된 device에 맞게 tensor 변환
        outputs = net(images) # 신경망에 이미지를 통과시켜 출력을 계산
        _, predicted = torch.max(outputs.data, 1) # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
correct_pred = {classname: 0 for classname in classes} # 각 분류(class)에 대한 예측값 계산을 위한 설정
total_pred = {classname: 0 for classname in classes}

with torch.no_grad(): # 학습과정이 아니므로 변화도는 여전히 필요하지 않습니다
    for data in testloader: #test data set에서 testloader롤 통한 data 불러오기
        images, labels = data[0].to(device), data[1].to(device)  #불러온 data inages, labels 에 지정 및 설정된 device에 맞게 tensor 변환
        outputs = net(images) # 신경망에 이미지를 통과시켜 출력을 계산
        _, predictions = torch.max(outputs, 1) # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택
        for label, prediction in zip(labels, predictions): # 각 분류별로 올바른 예측 수를 수집
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items(): # 각 분류별 정확도(accuracy)를 출력합니다
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, accuracy))